In [6]:
import os
import opendatasets as od

KAGGLE_URL = 'https://www.kaggle.com/datasets/mohit2512/jio-mart-product-items'
DATASET_PATH = '.\dataset'

def fetch_dataset(url = KAGGLE_URL, path = DATASET_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    od.download(url, path)

fetch_dataset()

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vladyslavachernorai
Your Kaggle Key: ········


100%|█████████████████████████████████████████████████████████████████████████████| 2.31M/2.31M [00:00<00:00, 4.29MB/s]


In [11]:
!pip install opendatasets

In [14]:
!pip install gensim

     ---------------------------------------- 24.0/24.0 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 983.8/983.8 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 56.8/56.8 kB 750.3 kB/s eta 0:00:00


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

DATASET_PATH = "dataset/jio-mart-product-items/jio_mart_items.csv"
mart_items = pd.read_csv(DATASET_PATH)

# train_set, test_set = train_test_split(mart_items, test_size=0.2, random_state=42)
mart_items.head(10)

,category,sub_category,href,items,price
0,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Fresh Dates (Pack) (Approx 450 g - 500 g),109.0
1,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Tender Coconut Cling Wrapped (1 pc) (Approx 90...,49.0
2,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Mosambi 1 kg,69.0
3,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Orange Imported 1 kg,125.0
4,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...,44.0
5,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Custard Apple 4 pcs (Approx 750 g - 1000 g),179.0
6,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Pineapple Queen 1 pc (Approx 700 g - 1200 g),84.0
7,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Dates Imported (Approx 400 g - 500 g),175.0
8,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Watermelon Kiran Big 1 pc (Approx. 2800 g - 40...,45.0
9,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Pear Green Indian 6 pcs (Approx 700 g - 1000 g),189.0


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

mart_items_processed = mart_items.dropna(subset=["items"])

# vectorizer = TfidfVectorizer()
# items_encoded = vectorizer.fit_transform(mart_items_processed['items']).toarray()
items = mart_items_processed['items'].values.copy()
items_processed = []
for i in range(len(items)):
  items_processed.append(items[i].split())

In [17]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

category_encoded = ordinal_encoder.fit_transform(mart_items_processed[['category']])
sub_category_encoded = ordinal_encoder.fit_transform(mart_items_processed[['sub_category']])


In [18]:
import numpy as np

price_values = mart_items_processed['price'].values
price_values = np.reshape(price_values, (len(price_values), 1))

In [ ]:
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# Create TaggedDocuments from the tokenized data
tagged_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(items_processed)]

# Train a Doc2Vec model
model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# input = np.concatenate((category_encoded, sub_category_encoded, price_values), axis=0)
# input = np.concatenate((input, items_encoded.toarray()), axis=0)

items_encoded = model.dv.vectors.copy()
num_rows, num_cols = items_encoded.shape
input_features = np.empty((num_rows, num_cols + 2))
for i in range(len(items_encoded)):
  input_features[i] = np.append(items_encoded[i], [sub_category_encoded[i], category_encoded[i]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(input_features, price_values, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = [{'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
              {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0], 'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]}]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = [{'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
              {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0], 'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]}]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(housing_prepared, housing_labels)